<a href="https://colab.research.google.com/github/david-meltzer/LLMs/blob/main/data_analysis/wiki_instructions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies



In [3]:
%cd drive/MyDrive/LLMs/wikipedia

/content/drive/MyDrive/LLMs/wikipedia


In [5]:
!pip install datasets --quiet
!pip install textstat --quiet
!pip install wandb --quiet
!pip install -U sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:0

In [8]:
from datasets import load_dataset, load_from_disk
from sentence_transformers import SentenceTransformer
from textstat import flesch_reading_ease as fre
from textstat import flesch_kincaid_grade as fkg

# Definitions

In [47]:
def flesch_scores(example):
    """
    Calculate Flesch Readability scores for each article
    """

    # Compute Flesch Readability score for each answer.
    fre_scores = [fre(text) for text in example['answers']['text']]
    # Compute Flesch Kincaid Grade level for each answer.
    fkg_scores = [fkg(text) for text in example['answers']['text']]
    # Add corresponding metrics to dictioanry example['answers'].
    example['answers']['fre'] = fre_scores
    example['answers']['fkg'] = fkg_scores

    return example

def extract_article_text(text):
    """
    Extracts the text from a Wikipedia article before the "Related pages" and "References" sections.

    Parameters:
        text (str): The entire Wikipedia article content as a string.

    Returns:
        str: The article text before the "Related pages" and "References" sections.
    """

    # Lower-case all text.
    text = text.lower()

    # Find the positions of the "Related pages" and "References" sections
    related_pages_position = text.find('Related pages')
    references_position = text.find('References')

    # Extract the text before the sections
    article_text = ""
    if related_pages_position != -1:
        # If the "Related pages" section is found, extract the text before it
        article_text = text[:related_pages_position]
    elif references_position != -1:
        # If the "References" section is found (but not the "Related pages" section),
        # extract the text before it
        article_text = text[:references_position]
    else:
        # If both sections are not present, return the entire text
        article_text = text

    # Remove leading and trailing whitespace from the extracted text
    return article_text.strip()

def remove_sections(article):

    article_split = article.split('\n')
    article_split = [txt for txt in article_split if len(txt.split())>=5]

    return '\n'.join(article_split)


In [ ]:
#import wandb
#run = wandb.init()
#artifact = run.use_artifact('ft-llmmm/distilgpt2_sw_blm/simplewiki_splits:v2', type='dataset')
#artifact_dir = artifact.download()

In [9]:
simple_wiki = load_from_disk('./artifacts/simplewiki_splits:v2/simplewiki')

In [26]:
simple_wiki['test'][1]['title']

'Battle of Monmouth'

In [33]:
simple_wiki['test'][1]['text'].lower().split('\n\n')[3]

'general george washington and general charles lee led the continental army to englishtown.  there, washington ordered the army to attack british general sir henry clinton\'s army.  during the battle, lee ordered his own continental troops to retreat.  washington became angry and ordered lee and "mad" anthony wayne to slow the british down while he, washington, took command.  the continental army defended its position while the british attacked all day.  that night, the british left in the dark.'

In [51]:
simple_wiki['test'][2]['title']

'Robinson Mill, California'

In [50]:
remove_sections(extract_article_text(simple_wiki['test'][2]['text'])).split('\n\n')

['robinson mill is a census-designated place (cdp) in butte county, california.\nsettlements in butte county, california']

In [ ]:
english_wiki = load_dataset("wikipedia", "20220301.en")

Downloading:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/20.3G [00:00<?, ?B/s]

# Scratch

In [ ]:
!pip install apache-beam
!pip install -U datasets
!pip install transformers[torch] evaluate
!pip install mwparserfromhell

In [ ]:
from datasets import (load_dataset,
                      load_from_disk,
                      concatenate_datasets)
import torch
from transformers import (AutoModelForCausalLM,
                          TrainingArguments,
                          Trainer,
                          AutoTokenizer,
                          DataCollatorForLanguageModeling)
import random

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
%ls

sample_data/


In [ ]:
%cd drive/MyDrive/LLMs/wikipedia

/content/drive/MyDrive/LLMs/wikipedia


In [ ]:
wiki_simple = load_from_disk('./data/wiki_simple')
wiki_split_tmp = wiki_simple.train_test_split(.1)
validation_dataset = wiki_split_tmp['test']
wiki_split = wiki_split_tmp['train'].train_test_split(test_size=len(wiki_split_tmp['test']))
wiki_split['validation'] = validation_dataset

In [ ]:
samp_train = random.sample(range(len(wiki_split_tmp['train'])),10)

In [ ]:
def trunc_article(text):
    text = text.split('\n')
    new_txt = text[0]
    i=1

    while len(new_txt)<=512 and i<len(text):
        new_txt += ' '+text[i]
        i += 1

    new_txt = ' '.join(new_txt.split())

    return new_txt.strip()

def create_prompt(text):
    prompt= f"Instruction: X\nOutput: {text}\nWhat kind of instruction, X, could this be an answer to?\nX:"
    return prompt

In [ ]:
trunc_articles_samp = list(map(trunc_article,wiki_simple['train'][samp_train]['text']))

In [ ]:
trunc_articles_samp[0]

'Abderrazak Rassaa (\u200e; 4 January 1930 – 7 January 2020) was a Tunisian politician. He was born in Tunis. He was a member of the Socialist Destourian Party. He served as Minister of Finance from 1969 to 1971. He was also Minister of Industry from 1968 to 1969. Rassaa died on 7 January 2020, at the age of 90. References 2020 deaths 1930 births People from Tunis Tunisian politicians'

In [ ]:
prompt = """Instruction {X}\nOutput: {}\nWhat kind of instruction, X, could this be the answer to?\
Write the instruction using Simple English.
X: """

In [ ]:
print(prompt)

Instruction {X}
Output: {}
What kind of instruction, X, could this be the answer to?Write the instruction using Simple English.
X: 
